<a href="https://colab.research.google.com/github/vannissimo/snake-ga/blob/master/pitch_control_acm_marcature2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
# install findspark using pip
!pip install -q findspark

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [5]:
import numpy as np
from bisect import bisect
from pyspark.sql import functions as func
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import concat,col,lit,desc
from operator import itemgetter
import itertools 
from bisect import bisect
import pandas as pd

In [6]:
team = team_a = 'milan'
opponent = 'cagliari'
half = 'first half'
half_split = '1'
player_in_possess = 'all'
secondi_di_differenza_giocatori_eventi = {'cagliari':-2}

name_path = team +"_"+str(opponent)+"_"+str(half).replace(' ','_')+"_"+str(half_split)+"_"+player_in_possess

name_path

'milan_cagliari_first_half_1_all'

In [7]:
# mount your drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# loading data as parquet
pre_data_players = spark.read.parquet("/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/frames_milan_cagliari.parquet").drop('index')

pre_pre_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/' + 'pitch_control_' + name_path + '.csv',header='True')
pre_pre_df = pre_pre_df.withColumn('x', pre_pre_df['x'].cast(DoubleType())).withColumn('y', pre_pre_df['y'].cast(DoubleType())).withColumn('target_y', pre_pre_df['target_y'].cast(DoubleType())).withColumn('target_x', pre_pre_df['target_x'].cast(DoubleType()))

pre_pre_df.where(pre_pre_df['target_x'] > -999).where(pre_pre_df['target_y'] < 9999).groupBy('time').agg(func.min('target_x'),func.max('target_x'),func.min('target_y'),func.max('target_y')).show()

+------+-------------+-------------+-------------+-------------+
|  time|min(target_x)|max(target_x)|min(target_y)|max(target_y)|
+------+-------------+-------------+-------------+-------------+
| 919.0|        -55.0|         54.0|        -35.0|         36.0|
|1362.0|        -55.0|         51.0|        -37.0|         36.0|
| 383.0|        -55.0|         27.0|        -37.0|         36.0|
| 609.0|        -55.0|         30.0|        -37.0|         36.0|
|1386.0|        -55.0|         54.0|        -37.0|         36.0|
|1198.0|        -55.0|         22.0|        -37.0|         36.0|
|1188.0|        -55.0|         35.0|        -37.0|         36.0|
| 138.0|        -55.0|         43.0|        -37.0|         36.0|
| 401.0|        -55.0|         23.0|        -37.0|         36.0|
| 289.0|        -55.0|         45.0|        -37.0|         36.0|
|  40.0|        -55.0|         45.0|        -30.0|         36.0|
+------+-------------+-------------+-------------+-------------+



In [9]:
pre_data_players.show(3)

+---------+----------+---------+------+----------+-----------+-------+---------+---------+--------------------+----------+-------------+
|      Vel|         x|        y|  Time|    player|       half|team_id|team_name|player_id|        italian_name|  position|real_position|
+---------+----------+---------+------+----------+-----------+-------+---------+---------+--------------------+----------+-------------+
|      0.0|       0.9|     33.2|5594.5|Castillejo|second half|   t120|    Milan|  p193449|samuel castillejo...|Midfielder|       Winger|
| 2.022165|0.83794284|     32.8|5594.7|Castillejo|second half|   t120|    Milan|  p193449|samuel castillejo...|Midfielder|       Winger|
|2.2488973|       0.8|32.578335|5594.8|Castillejo|second half|   t120|    Milan|  p193449|samuel castillejo...|Midfielder|       Winger|
+---------+----------+---------+------+----------+-----------+-------+---------+---------+--------------------+----------+-------------+
only showing top 3 rows



In [10]:
#create parameters input boxes (Databricks)

if opponent == 'milan':
  opponent_f = team_a
else:
  opponent_f = opponent

In [11]:
from pyspark.sql import functions as func

pre_data_players = pre_data_players.withColumnRenamed('team_name','team').withColumnRenamed('position','role')

min_half_time, max_half_time = pre_data_players.where(pre_data_players['half'] == half).groupBy('half').agg(func.min('time'),func.max('time')).collect()[0][1:]

if half_split == '1':
  data_players = pre_data_players.where((pre_data_players['time'] > min_half_time) & (pre_data_players['time'] <= min_half_time+float((max_half_time-min_half_time)/2.0)))
  
elif half_split == '2':
  data_players = pre_data_players.where((pre_data_players['time'] > min_half_time+float((max_half_time-min_half_time)/2.0)) & (pre_data_players['time'] < max_half_time))


In [12]:
from operator import itemgetter
from pyspark.sql import functions as func
from pyspark.sql.functions import lit

import itertools 
import bisect
import numpy as np
from pyspark.sql.types import DoubleType
import pandas as pd

team_a_lower = team_a.lower()
granularity = 0.5
max_ball_distance = 70.0
max_meters_distance_player_can_receive_ball = 30.0
opponent_lower = opponent.lower()

team_a_goalies = data_players.where(data_players['team'] == team_a.title()).where(data_players['role'] == 'Goalkeeper').select('player_id').distinct().collect()[0]['player_id']
opponent_goalies = data_players.where(data_players['team'] == opponent.title()).where(data_players['role'] == 'Goalkeeper').select('player_id').distinct().collect()[0]['player_id']

goalie_x = data_players.where(data_players['team'] == team_a.title()).where(data_players['role'] == 'Goalkeeper').groupBy('role').avg('x').rdd.max()[1]

team_a_goalies,opponent_goalies,goalie_x

('p129487', 'p213535', 33.22888834675131)

In [13]:
min_x = data_players.select('x').rdd.min()[0]
max_x = data_players.select('x').rdd.max()[0]
min_y = data_players.select('y').rdd.min()[0]
max_y = data_players.select('y').rdd.max()[0]

x_lines = list(np.arange(int(min_x)-1,int(max_x)+1,float(max_x-min_x)/6.0))
y_lines = list(np.arange(int(min_y)-1,int(max_y)+1,float(max_y-min_y)/4.0))

mid_x, mid_y = x_lines[3], float(y_lines[2])

if float(np.abs(goalie_x - max_x)) < float(np.abs(goalie_x - min_x)):
  verse = 'right_to_left'
  opponent_verse = 'left_to_right'
else:
  verse = 'left_to_right'
  opponent_verse = 'right_to_left'

team_a_goalies,verse

('p129487', 'right_to_left')

In [14]:
pre_pre_df = pre_pre_df.withColumn('time', pre_pre_df['time'].cast(DoubleType()))

In [15]:
possess = pre_pre_df.where(pre_pre_df['target_x'] < -950).select('player_id','team','time','x','y').distinct().withColumnRenamed('player_id','player_in_possess').withColumnRenamed('team','team_in_possess')

timings = sorted(possess.select('time').rdd.map(lambda a : a[0]).collect())
timings

[40.0,
 138.0,
 289.0,
 383.0,
 401.0,
 609.0,
 919.0,
 1188.0,
 1198.0,
 1362.0,
 1386.0]

In [16]:
pre_pre_df = pre_pre_df.where(pre_pre_df['target_x'] > -999)
df = pre_pre_df.distinct()

In [17]:
pre_pre_df.groupBy('team').count().show()

+--------+-----+
|    team|count|
+--------+-----+
|    null|    1|
|   Milan|57611|
|Cagliari|47977|
+--------+-----+



In [18]:
#offside_players = pre_df.where(pre_df['team'] == team_a.capitalize()).select('player_id','time','offside_flag').distinct()

In [19]:
verse

'right_to_left'

In [20]:
df_locs = df.select('player_id','time','team','x','y').distinct()

if verse == 'right_to_left':
  opponent_goal_line = [min_x+5.0, float(max_y+min_y)/2.0]  
  team_a_goal_line = [max_x-5.0, float(max_y+min_y)/2.0]
else:
  opponent_goal_line = [max_x-5.0, float(max_y+min_y)/2.0]
  team_a_goal_line = [min_x+5.0, float(max_y+min_y)/2.0]  
  
df_locs = df_locs.withColumn('distance_from_opponent_goal_line', 0.25*func.sqrt((df_locs['x'] - opponent_goal_line[0])**2 + (df_locs['y'] - opponent_goal_line[1])**2) + 0.75*func.sqrt((df_locs['x'] - opponent_goal_line[0])**2))
df_locs = df_locs.withColumn('distance_from_team_goal_line', 0.25*func.sqrt((df_locs['x'] - team_a_goal_line[0])**2 + (df_locs['y'] - team_a_goal_line[1])**2) + 0.75*func.sqrt((df_locs['x'] - team_a_goal_line[0])**2))

panda_locs = df_locs.toPandas()

verse,opponent_goal_line

('right_to_left', [-51.5, -7.98222541809082])

In [21]:
panda_possess = possess.toPandas()

In [22]:
x_ranges = list(np.arange(min_x,max_x,3.0))
y_ranges = list(np.arange(min_y,max_y,3.0))

In [23]:
x_ranges, y_ranges

([-56.5,
  -53.5,
  -50.5,
  -47.5,
  -44.5,
  -41.5,
  -38.5,
  -35.5,
  -32.5,
  -29.5,
  -26.5,
  -23.5,
  -20.5,
  -17.5,
  -14.5,
  -11.5,
  -8.5,
  -5.5,
  -2.5,
  0.5,
  3.5,
  6.5,
  9.5,
  12.5,
  15.5,
  18.5,
  21.5,
  24.5,
  27.5,
  30.5,
  33.5,
  36.5,
  39.5,
  42.5,
  45.5,
  48.5,
  51.5,
  54.5,
  57.5,
  60.5],
 [-55.0,
  -52.0,
  -49.0,
  -46.0,
  -43.0,
  -40.0,
  -37.0,
  -34.0,
  -31.0,
  -28.0,
  -25.0,
  -22.0,
  -19.0,
  -16.0,
  -13.0,
  -10.0,
  -7.0,
  -4.0,
  -1.0,
  2.0,
  5.0,
  8.0,
  11.0,
  14.0,
  17.0,
  20.0,
  23.0,
  26.0,
  29.0,
  32.0,
  35.0,
  38.0])

In [24]:
def x_area_handling(x):
  
  if x >= 30:
    return x
  elif x < 30 and x >= 10:
    return int(0.8*float(x))
  elif x < 10 and x >= -20:
    return int(0.5*float(x))
  elif x < -20:
    return 10

x_single_area_prob = 100.0/float(len(x_ranges))
y_single_area_prob = 100.0/float(len(y_ranges))

x_ranges_value_right_to_left = list(map(lambda d : x_area_handling(float(x_single_area_prob)*(len(x_ranges)-x_ranges.index(d))), x_ranges))
x_ranges_value_left_to_right = list(map(lambda d : x_area_handling(float(x_single_area_prob)*(x_ranges.index(d))), x_ranges))
y_ranges_value = list(map(lambda d : 100-1.75*(float(y_single_area_prob)*float(np.abs((mid_y-y_ranges.index(d))))), y_ranges))
                      
if verse == 'left_to_right':
  x_ranges_value = x_ranges_value_left_to_right
  opp_x_ranges_value = x_ranges_value_right_to_left
elif verse == 'right_to_left':
  x_ranges_value = x_ranges_value_right_to_left
  opp_x_ranges_value = x_ranges_value_left_to_right
  
x_ranges_value,y_ranges_value

([100.0,
  97.5,
  95.0,
  92.5,
  90.0,
  87.5,
  85.0,
  82.5,
  80.0,
  77.5,
  75.0,
  72.5,
  70.0,
  67.5,
  65.0,
  62.5,
  60.0,
  57.5,
  55.0,
  52.5,
  50.0,
  47.5,
  45.0,
  42.5,
  40.0,
  37.5,
  35.0,
  32.5,
  30.0,
  22,
  20,
  18,
  16,
  14,
  12,
  10,
  8,
  3,
  2,
  1],
 [50.878454744815826,
  45.409704744815826,
  39.940954744815826,
  34.472204744815826,
  29.003454744815826,
  23.534704744815826,
  18.065954744815826,
  12.597204744815826,
  7.128454744815826,
  1.6597047448158264,
  -3.8090452551841736,
  -9.277795255184174,
  -14.746545255184174,
  -20.215295255184174,
  -25.684045255184174,
  -31.152795255184174,
  -36.621545255184174,
  -42.090295255184174,
  -47.559045255184174,
  -53.027795255184174,
  -58.496545255184174,
  -63.965295255184174,
  -69.43404525518417,
  -74.90279525518417,
  -80.37154525518417,
  -85.84029525518417,
  -91.30904525518417,
  -96.77779525518417,
  -102.24654525518417,
  -107.71529525518417,
  -113.18404525518417,
  -118.65

In [53]:
def probabilityPlusMinus(x):
  
  if x[6] == opponent.title():
    return list(x[:3])+[-float(x[3])]+list(x[4:])+[opponent_verse]
  else:
    return list(x)+[verse]
  
def opponentCount(x):
  
  try:
    team_1 = float(x[3])
  except:
    team_1 = 0.0

  try:
    team_2 = float(x[5])
  except:
    team_2 = 0.0

  try:
    opp_1 = int(x[4])
  except:
    opp_1 = 0

  try:
    opp_2 = int(x[6])
  except:
    opp_2 = 0

  if team_1 > team_2:
    return [x[0],x[1],x[2],x[3],-opp_2]
  else:
    return [x[0],x[1],x[2],x[5],-opp_1]

df = df.withColumn('probability', df['probability'].cast(DoubleType()))

pre_max_pitch_pivot_gr = df.where(df['team'] != '').groupBy('target_x','target_y','time').pivot('team').agg(func.max('probability'),func.count('player_id'))

max_pitch_pivot = spark.createDataFrame(pre_max_pitch_pivot_gr.rdd.map(lambda a : opponentCount(a)),['target_x','target_y','time','probability','opponents_count'])

max_pitch_pivot.show()

+--------+--------+------+------------------+---------------+
|target_x|target_y|  time|       probability|opponents_count|
+--------+--------+------+------------------+---------------+
|   -21.0|    15.0|1188.0|             100.0|              0|
|    35.0|    -5.0|1362.0|             100.0|              0|
|   -40.0|    14.0|1362.0|             100.0|              0|
|   -50.0|    14.0|1386.0|             100.0|              0|
|   -55.0|    -2.0| 289.0|             100.0|              0|
|    43.0|   -10.0| 919.0|             100.0|              0|
|   -34.0|    25.0| 289.0|58.961500173406854|             -1|
|   -48.0|    -3.0| 609.0|             100.0|              0|
|    27.0|     8.0| 289.0|             100.0|              0|
|   -43.0|    -2.0| 383.0| 57.51929834242605|             -1|
|   -43.0|   -28.0| 289.0|54.795003931509854|             -1|
|   -14.0|   -18.0|  40.0|             100.0|              0|
|   -38.0|    34.0|  40.0|             100.0|              0|
|    26.

In [54]:
max_player = df.select('target_x','target_y','time','player_id','team','probability','distance_from_target','x','y','slope_fuzzy_compare','ball_distance')

In [56]:
max_pitch_pivot_j = max_pitch_pivot.join(max_player, ['target_x','target_y','time','probability'])

max_pitch_pivot_jj = spark.createDataFrame(max_pitch_pivot_j.rdd.map(lambda f : probabilityPlusMinus(f)),max_pitch_pivot_j.columns+['verse'])

pre_max_pitch_pivot_jjj = max_pitch_pivot_jj.where(max_pitch_pivot_jj['team'] == opponent.capitalize())

pre_max_pitch_pivot_jjj.show()

+--------+--------+------+-------------------+---------------+----------+--------+--------------------+-------------------+-------------------+-------------------+------------------+-------------+
|target_x|target_y|  time|        probability|opponents_count| player_id|    team|distance_from_target|                  x|                  y|slope_fuzzy_compare|     ball_distance|        verse|
+--------+--------+------+-------------------+---------------+----------+--------+--------------------+-------------------+-------------------+-------------------+------------------+-------------+
|   -40.0|    14.0|1362.0|             -100.0|              0| Radunovic|Cagliari|  11.148897425654473|-42.400001525878906| 3.1124885082244873|                3.0|36.645869481600556|left_to_right|
|   -50.0|    14.0|1386.0|             -100.0|              0| Radunovic|Cagliari|  17.801311065594803|-38.920509338378906|0.06688971072435379|                5.0| 37.07748181097742|left_to_right|
|   -55.0|    -

In [57]:
max_pitch_pivot_jj.groupBy('player_id','team').avg('probability').show()

+------------+--------+-------------------+
|   player_id|    team|   avg(probability)|
+------------+--------+-------------------+
|      Deiola|Cagliari| -58.36165706554735|
|       Kjaer|   Milan|  84.27528108246118|
|    Calabria|   Milan|  76.35664046939465|
|       Marin|Cagliari| -65.17654706164488|
|   Strootman|Cagliari|-53.804130769211284|
|      Giroud|   Milan|  46.97687786910945|
|        Diaz|   Milan|  60.05566957626454|
|        Leao|   Milan|  60.47050201840573|
|   Pavoletti|Cagliari| -66.07690771291229|
|     Carboni|Cagliari| -70.10097504890899|
|      Krunic|   Milan|  63.11642807985858|
|      Tonali|   Milan|   66.4154104679352|
|       Godin|Cagliari| -67.04006366585699|
|      Tomori|   Milan|  80.25961145599501|
|  Joao Pedro|Cagliari| -71.61207529430531|
|Saelemaekers|   Milan|  67.69994393799074|
|     Dalbert|Cagliari| -83.00680359403158|
|  Ceppitelli|Cagliari| -50.03262053312109|
|     Maignan|   Milan|  95.18536110229726|
|      Nandez|Cagliari| -79.8252

In [58]:
pd_probs = pre_max_pitch_pivot_jjj.toPandas()

In [59]:
def findBestMaxObs(x,panda,max_obs):
  
  probs = panda[panda['player_id'] == x[0]][panda['time'] == x[1]][['probability','distance_from_target']].values.tolist()
  
  try:
    first_twenty_probs = sorted(list(map(lambda a : a[0],probs)), reverse = True)[:max_obs]
    prob_twentieth = first_twenty_probs[-1]
    distance_twentieth = 100.0

    if float(prob_twentieth) == 100.0:
      first_twenty_distances = sorted(list(map(lambda g : g[1], list(filter(lambda a : a[0] == 100.0,probs)))))[:max_obs]
      distance_twentieth = first_twenty_distances[-1]

    return [prob_twentieth,distance_twentieth]
  except:
    return [0.0,0.0]

players_times_values = spark.createDataFrame(df.where(df['team'] == opponent.title()).select('player_id','time').distinct().rdd.map(lambda a : list(a)+findBestMaxObs(a,pd_probs,60)),['player_id','time','probability_twentieth','distance_twentieth'])

players_times_values.show()

+----------+------+---------------------+------------------+
| player_id|  time|probability_twentieth|distance_twentieth|
+----------+------+---------------------+------------------+
| Pavoletti| 919.0|   -66.31318955891913|             100.0|
|   Carboni|1188.0|   -42.78363803363206|             100.0|
| Radunovic| 919.0|   -53.81953513499719|             100.0|
|    Deiola| 289.0|               -100.0|             100.0|
| Strootman| 383.0|  -37.003529119643865|             100.0|
|   Carboni| 289.0|               -100.0|             100.0|
|   Carboni| 138.0|               -100.0|             100.0|
|   Dalbert| 383.0|  -52.339009366953135|             100.0|
|    Deiola|1362.0|  -45.678376099534795|             100.0|
|Ceppitelli|1188.0|  -36.853391542485355|             100.0|
|     Godin| 138.0|               -100.0|             100.0|
|    Deiola|1188.0|    -62.1592515499886|             100.0|
| Radunovic| 289.0|   -44.34850262780635|             100.0|
|     Godin|1198.0|   -4

In [60]:
pd_to_filter = players_times_values.toPandas()

In [61]:
pre_max_pitch_pivot_jjj

DataFrame[target_x: double, target_y: double, time: double, probability: double, opponents_count: bigint, player_id: string, team: string, distance_from_target: string, x: double, y: double, slope_fuzzy_compare: string, ball_distance: string, verse: string]

In [62]:
def obsToFilter(x, panda):
  
  prob_filter, distance_filter = panda[panda['player_id'] == x[5]][panda['time'] == x[2]][['probability_twentieth','distance_twentieth']].values.tolist()[0]
  
  flag = ''
  
  if (float(x[3]) < prob_filter) or (float(x[7]) > distance_filter):
    flag = 'out'
  
  return flag
  
max_pitch_pivot_jjj_pre_filt = spark.createDataFrame(pre_max_pitch_pivot_jjj.rdd.map(lambda a : list(a)+[obsToFilter(a, pd_to_filter)]), pre_max_pitch_pivot_jjj.columns+['obs_flag'])

max_pitch_pivot_jjj = max_pitch_pivot_jjj_pre_filt.where(max_pitch_pivot_jjj_pre_filt['obs_flag'] == '').where(max_pitch_pivot_jjj_pre_filt['probability'] > 0).drop('obs_flag')

In [63]:
max_pitch_pivot_jjj.show()

+--------+--------+----+-----------+---------------+---------+----+--------------------+---+---+-------------------+-------------+-----+
|target_x|target_y|time|probability|opponents_count|player_id|team|distance_from_target|  x|  y|slope_fuzzy_compare|ball_distance|verse|
+--------+--------+----+-----------+---------------+---------+----+--------------------+---+---+-------------------+-------------+-----+
+--------+--------+----+-----------+---------------+---------+----+--------------------+---+---+-------------------+-------------+-----+



In [ ]:
#display(max_pitch_pivot_jjj.where(max_pitch_pivot_jjj['target_x'] == 42.0).where(max_pitch_pivot_jjj['target_y'] == 5.0).where(max_pitch_pivot_jjj['time'] == 1135.0))

In [64]:
from pyspark.sql.functions import lit

def ifHigherThanLimit(a,limit,maxMinFlag): 
  if maxMinFlag == 'max':
    return float(np.min([limit,a]))
  elif maxMinFlag == 'min':
    return float(np.max([limit,a]))

def addFakeProb(team):
  
  if team == opponent.title():
    return 150.0
  else:
    return -150.0

  
### PITCH ###
pitch_pivot = max_pitch_pivot_jj.select('target_x','target_y','time','probability','player_id','team')
pitch_pivot = pitch_pivot.withColumn('type',lit('pitch'))
pitch_pivot = pitch_pivot.withColumnRenamed('target_x','x').withColumnRenamed('target_y','y')

### PLAYERS ###
players_locs = spark.createDataFrame(data_players.select('player_id','team','time','x','y').withColumn('type',lit('player')).distinct().rdd.map(lambda a : list(a)+[addFakeProb(a[1])]), ['player_id','team','time','x','y','type','probability'])
players_locs = players_locs.where(players_locs['time'].isin(timings))

### BALL ###
ball = pre_pre_df.where(pre_pre_df['target_x'] == -999).orderBy('time')
ball_locs = ball.withColumn('probability',lit(200.0)).withColumn('type',lit('ball')).select('x','y','time','probability','player_id','team','type')
pd_ball = ball.toPandas()

In [65]:
ball_locs.show()

+---+---+----+-----------+---------+----+----+
|  x|  y|time|probability|player_id|team|type|
+---+---+----+-----------+---------+----+----+
+---+---+----+-----------+---------+----+----+



In [66]:
def addNegativeProb(x):
  
  team = x[team_index]
  probability = x[prob_index]
  
  if team == team_a.title():
    new_prob = -float(probability)
  else:
    new_prob = float(probability)
    
  y = list(x[:prob_index])+[new_prob]+list(x[1+prob_index:])
  
  return y
  
pitch_pivot_union = pitch_pivot.distinct()

team_index, prob_index = pitch_pivot_union.columns.index('team'),pitch_pivot_union.columns.index('probability')

pitch_pivot_union = spark.createDataFrame(pitch_pivot_union.rdd.map(lambda y : addNegativeProb(y)), pitch_pivot_union.columns)

for i in players_locs.columns:
  if 'class' in str(i):
    pitch_pivot_union = pitch_pivot_union.withColumn(str(i), lit(''))
    
out = pitch_pivot_union.union(players_locs.select(pitch_pivot_union.columns)).union(ball_locs.select(pitch_pivot_union.columns)).withColumnRenamed('player_id','player')

In [67]:
out.count()

65157

In [69]:
new_path = 'pitch_control_2_'+name_path
new_path

'pitch_control_2_milan_cagliari_first_half_1_all'

In [70]:
out.coalesce(1).write.option('delimiter',',').option('header','true').csv('/content/drive/MyDrive/Colab Notebooks/FootballDataAnalysis/data/Milan-Cagliari/parquet/'+new_path+'.csv', mode='overwrite')

In [47]:
pd_pitch = max_pitch_pivot_jj.toPandas()

In [ ]:
from bisect import bisect
from pyspark.sql.window import Window

def areaDanger(d,x_ranges,y_ranges):
  
  if d[-1] == 'right_to_left':
    x_values = x_ranges_value_right_to_left
  else:
    x_values = x_ranges_value_left_to_right
  
  try:
    x_value = x_values[bisect(x_ranges,d[0])]
  except:
    try:
      x_value = x_values[bisect(x_ranges,d[0])-1]
    except:
      x_value = x_values[bisect(x_ranges,d[0])+1]
      
  try:
    y_value = y_ranges_value[bisect(y_ranges,d[1])]
  except:
    try:
      y_value = y_ranges_value[bisect(y_ranges,d[1])-1]
    except:
      y_value = y_ranges_value[bisect(y_ranges,d[1])+1]
  
  if x_value < 50:
    y_value = 10
  
  return [bisect(x_ranges,d[0]),bisect(y_ranges,d[1]),float(np.round(x_value,0)),float(np.round(y_value,0)),float(np.round(x_value+y_value,0))]
 
def findOpponentsOvercomeByThePass(x,panda,panda_possess,pd_pitch):
  
  player_x = x[x_index]
  player_y = x[y_index]
  verse = x[verse_index]
  time_ = x[time_index]
  target_x = x[target_x_index]
  target_y = x[target_y_index]
  
  player_possess,possess_x,possess_y = panda_possess[panda_possess['time'] == time_][['player_in_possess','x','y']].values.tolist()[0]
  
  sit_x = 0.25*player_x+0.75*target_x
  sit_y = 0.25*player_y+0.75*target_y

  x_advancement_flag = float(np.sign(team_a_goal_line[0] - target_x))

  values_x = [sit_x-(x_advancement_flag*10),sit_x+(x_advancement_flag*10)]

  back_min_x,ahead_max_x = float(np.min(values_x)),float(np.max(values_x))

  up_y = sit_y+10
  down_y = sit_y-10

  quarters_control_percentages = []

  close_opponents_panda = panda[panda['team'] == team_a.title()][panda['time'] == time_][panda['x'] > player_x-4.0][panda['x'] < player_x+4.0][panda['y'] > player_y-4.0][panda['x'] < player_y+4.0]
  close_opponents = len(close_opponents_panda)
  opponents_x_y = panda[panda['team'] == team_a.title()][panda['time'] == time_][panda['player_id'] != team_a_goalies][['x','y']].values.tolist()
  
  close_opponents_distances = []
  for j in opponents_x_y:
    close_opponents_distances.append(float(np.sqrt((player_x-j[0])**2+(player_y-j[1])**2)))
     
  closest_opponent_distance = sorted(close_opponents_distances)[0]
  player_distance_from_goal_line = float(np.sqrt((player_x - opponent_goal_line[0])**2 + (player_y - opponent_goal_line[1])**2))
  
  opponents_distances_to_goal_line = []
  for j in opponents_x_y:
    opponents_distances_to_goal_line.append(float(np.sqrt((opponent_goal_line[0]-j[0])**2+(opponent_goal_line[1]-j[1])**2)))
    
  opponents_from_goal_line = len(list(filter(lambda a : a < player_distance_from_goal_line-1.0, opponents_distances_to_goal_line)))
    
  return [closest_opponent_distance,opponents_from_goal_line,close_opponents,player_distance_from_goal_line]
  
verse_index = max_pitch_pivot_jjj.columns.index('verse')
time_index = max_pitch_pivot_jjj.columns.index('time')
target_x_index = max_pitch_pivot_jjj.columns.index('target_x')
target_y_index = max_pitch_pivot_jjj.columns.index('target_y')
y_index = max_pitch_pivot_jjj.columns.index('y')
x_index = max_pitch_pivot_jjj.columns.index('x')

max_pitch_pivot_jjj = max_pitch_pivot_jjj.withColumn('target_x', max_pitch_pivot_jjj['target_x'].cast(DoubleType())).withColumn('target_y', max_pitch_pivot_jjj['target_y'].cast(DoubleType())).withColumn('x', max_pitch_pivot_jjj['x'].cast(DoubleType())).withColumn('y', max_pitch_pivot_jjj['y'].cast(DoubleType()))

#ppp_area_targets = spark.createDataFrame(max_pitch_pivot_jjj.where((max_pitch_pivot_jjj['player_id'].isin([team_a_goalies])) == False).rdd.map(lambda g : list(g)+areaDanger(g,x_ranges,y_ranges)+findOpponentsOvercomeByThePass(g,panda_locs,panda_possess,pd_pitch)), max_pitch_pivot_jjj.columns+['x_area','y_area','x_value','y_value','area_value','closest_opponent_distance','opponents_from_goal_line','close_opponents','player_distance_from_goal_line'])

#area_targets_advance_pre_final = ppp_area_targets.select('target_x','target_y','player_id','time','area_value','ball_distance','closest_opponent_distance','opponents_from_goal_line','distance_from_target','x','y','probability','close_opponents','player_distance_from_goal_line','slope_fuzzy_compare').distinct()

max_pitch_pivot_jjj.show()


In [ ]:
display(area_targets_advance_pre_final.where(area_targets_advance_pre_final['time'] == 3981))

In [ ]:
def singlePointOpponents(x,y,panda_locs,opponent,time_,meters_to_intercept,possess_x,possess_y):
  
  distance = distance = float(np.sqrt(((x-possess_x)**2)+((y-possess_y)**2)))
  
  opponent_meters_run = 3*(float(distance)/18.0)
  opps =  panda_locs[panda_locs['team'] == opponent.title()][panda_locs['time'] == time_][panda_locs['x'] > x-meters_to_intercept-opponent_meters_run][panda_locs['x'] < x+meters_to_intercept+opponent_meters_run][panda_locs['y'] > y-meters_to_intercept-opponent_meters_run][panda_locs['y'] < y+meters_to_intercept+opponent_meters_run]['player_id'].values.tolist()
  
  try:
    return opps[0]
  except:
    return []
  
def interceptionOpponents(player,time_,target_x,target_y,x,y,panda,panda_possess):
  
  player_possess,possess_x,possess_y = panda_possess[panda_possess['time'] == time_][['player_in_possess','x','y']].values.tolist()[0]

  interception_opponents = []
  interception_opponents_count = len(interception_opponents)
  
  if str(player) == str(player_possess):
    pass
    
  else:
    distance = float(np.sqrt(((target_x-possess_x)**2)+((target_y-possess_y)**2)))

    m = (possess_y-target_y)/(possess_x-target_x)

    b = (possess_x*target_y - target_x*possess_y)/(possess_x-target_x)

    if possess_x > target_x:
      xs = sorted(list(np.arange(target_x,possess_x,float(possess_x-target_x)/distance)), reverse = True)
    else:
      xs = list(np.arange(possess_x,target_x,float(target_x-possess_x)/distance))

    pre_trajectory = list(map(lambda xx: [float(xx),float((m*xx)+b)], xs))

    if distance > 25:
      trajectory = [pre_trajectory[2],pre_trajectory[3],pre_trajectory[4],pre_trajectory[5],pre_trajectory[6],pre_trajectory[-6],pre_trajectory[-5],pre_trajectory[-4],pre_trajectory[-3],pre_trajectory[-2],pre_trajectory[-1]]
    else:
      trajectory = pre_trajectory[2:]

    interception_opponents = list(filter(lambda a : a!= [], list(map(lambda d : singlePointOpponents(d[0],d[1],panda,opponent,time_,1.0,possess_x,possess_y), trajectory))))
    interception_opponents_count = len(list(dict.fromkeys(interception_opponents)))
    
  return [interception_opponents_count]


player_index = area_targets_advance_pre_final.columns.index('player_id')
time_index = area_targets_advance_pre_final.columns.index('time')
target_x_index = area_targets_advance_pre_final.columns.index('target_x')
target_y_index = area_targets_advance_pre_final.columns.index('target_y')
x_index = area_targets_advance_pre_final.columns.index('x')
y_index = area_targets_advance_pre_final.columns.index('y')

area_targets_advance_pre_final_intercept = sqlContext.createDataFrame(area_targets_advance_pre_final.rdd.map(lambda g : list(g)+interceptionOpponents(g[player_index],g[time_index],g[target_x_index],g[target_y_index],g[x_index],g[y_index],panda_locs,panda_possess)), area_targets_advance_pre_final.columns+['interception_opponents'])


In [ ]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

out.write.saveAsTable("pitch_control_2_"+str(team_a_lower)+"_"+str(opponent_lower)+"_"+str(half).replace(' ','_')+str(half_split)+"_"+str(player_in_possess), mode = 'overwrite') 

In [ ]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

area_targets_advance_pre_final_intercept.write.saveAsTable("kpis_marcature_"+str(team_a_lower)+"_"+str(opponent_lower)+"_"+str(half).replace(' ','_')+str(half_split)+"_"+str(player_in_possess), mode = 'overwrite') 

In [ ]:
str(half).replace(' ','_')+str(half_split)

In [ ]:
'''
# area_value
area_targets_advance_pre_final_area_value = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('area_value'),func.max('area_value')).withColumnRenamed('avg(area_value)','avg').withColumnRenamed('max(area_value)','max').withColumn('kpi', lit('area_value'))
area_targets_advance_pre_final_area_value = area_targets_advance_pre_final_area_value.withColumn('area_value_index', func.log((1+(0.45*area_targets_advance_pre_final_area_value['avg']+0.55*area_targets_advance_pre_final_area_value['max']))/area_targets_advance_pre_final_area_value['avg'])).select('player_id','time','area_value_index')
# opponents_cut
area_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('opponents_cut'),func.max('opponents_cut')).withColumnRenamed('avg(opponents_cut)','avg').withColumnRenamed('max(opponents_cut)','max').withColumn('kpi', lit('opponents_cut'))
area_targets_advance_pre_final_opponents_cut = area_targets_advance_pre_final_opponents_cut.withColumn('opponents_cut_index', func.log((1+(0.45*area_targets_advance_pre_final_opponents_cut['avg']+0.55*area_targets_advance_pre_final_opponents_cut['max']))/area_targets_advance_pre_final_opponents_cut['avg'])).select('player_id','time','opponents_cut_index')
# percentage_zone_in_control
area_targets_advance_pre_final_percentage_zone_in_control = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('percentage_zone_in_control'),func.max('percentage_zone_in_control')).withColumnRenamed('avg(percentage_zone_in_control)','avg').withColumnRenamed('max(percentage_zone_in_control)','max').withColumn('kpi', lit('percentage_zone_in_control'))
area_targets_advance_pre_final_percentage_zone_in_control = area_targets_advance_pre_final_percentage_zone_in_control.withColumn('percentage_zone_in_control_index', func.log((1+(0.45*area_targets_advance_pre_final_percentage_zone_in_control['avg']+0.55*area_targets_advance_pre_final_percentage_zone_in_control['max']))/area_targets_advance_pre_final_percentage_zone_in_control['avg'])).select('player_id','time','percentage_zone_in_control_index')
# team_advancement
area_targets_advance_pre_final_team_advancement = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('team_advancement'),func.max('team_advancement')).withColumnRenamed('avg(team_advancement)','avg').withColumnRenamed('max(team_advancement)','max').withColumn('kpi', lit('team_advancement'))
area_targets_advance_pre_final_team_advancement = area_targets_advance_pre_final_team_advancement.withColumn('team_advancement_index', func.log((1+(0.45*area_targets_advance_pre_final_team_advancement['avg']+0.55*area_targets_advance_pre_final_team_advancement['max']))/area_targets_advance_pre_final_team_advancement['avg'])).select('player_id','time','team_advancement_index')
# distance_from_target
area_targets_advance_pre_final_distance_from_target = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('distance_from_target'),func.max('distance_from_target')).withColumnRenamed('avg(distance_from_target)','avg').withColumnRenamed('max(distance_from_target)','max').withColumn('kpi', lit('distance_from_target'))
area_targets_advance_pre_final_distance_from_target = area_targets_advance_pre_final_distance_from_target.withColumn('distance_from_target_index', func.log((1+(0.45*area_targets_advance_pre_final_distance_from_target['avg']+0.55*area_targets_advance_pre_final_distance_from_target['max']))/area_targets_advance_pre_final_distance_from_target['avg'])).select('player_id','time','distance_from_target_index')
# probability
area_targets_advance_pre_final_probability = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('probability'),func.max('probability')).withColumnRenamed('avg(probability)','avg').withColumnRenamed('max(probability)','max').withColumn('kpi', lit('probability'))
area_targets_advance_pre_final_probability = area_targets_advance_pre_final_probability.withColumn('probability_index', func.log((1+(0.45*area_targets_advance_pre_final_probability['avg']+0.55*area_targets_advance_pre_final_probability['max']))/area_targets_advance_pre_final_probability['avg'])).select('player_id','time','probability_index')
# ball_distance
area_targets_advance_pre_final_ball_distance = area_targets_advance_pre_final.groupBy('player_id','time').agg(func.avg('ball_distance'),func.max('ball_distance')).withColumnRenamed('avg(ball_distance)','avg').withColumnRenamed('max(ball_distance)','max').withColumn('kpi', lit('ball_distance'))
area_targets_advance_pre_final_ball_distance = area_targets_advance_pre_final_ball_distance.withColumn('ball_distance_index', func.log((1+(0.45*area_targets_advance_pre_final_ball_distance['avg']+0.55*area_targets_advance_pre_final_ball_distance['max']))/area_targets_advance_pre_final_ball_distance['avg'])).select('player_id','time','ball_distance_index')

kpis = area_targets_advance_pre_final_area_value.join(area_targets_advance_pre_final_opponents_cut, ['player_id','time']).join(area_targets_advance_pre_final_percentage_zone_in_control, ['player_id','time']).join(area_targets_advance_pre_final_team_advancement, ['player_id','time']).join(area_targets_advance_pre_final_distance_from_target, ['player_id','time']).join(area_targets_advance_pre_final_probability, ['player_id','time']).join(area_targets_advance_pre_final_ball_distance, ['player_id','time'])
'''